<a href="https://colab.research.google.com/github/Scott-Ramirez/Comandos-Basicos-de-GIT/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import json as js
import nltk as nltk
import pickle as pk
import random as random

from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers import SGD 

lematizer = nltk.stem.WordNetLemmatizer()

#leer archivo json
intents = js.loads(open("/content/data.json").read())
#creacion de entidades de la lista de los datos
palabras = []
clases = []
documentos = []

ignorar_letras = ["?","!","ñ",","]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #separar las palabras para patterns
    lista_palabras=nltk.word_tokenize(pattern)
    palabras.extend(lista_palabras)
    #asociar patterns a los tags respectivos
    documentos.append(((lista_palabras),intent['tag']))
    #agregar los tags a las clases
    if intent['tag'] not in clases:
      clases.append(intent['tag'])
#almacenar las palabras raiz o lemas
palabras_ = [lematizer.lemmatize(palabras_local)
            for palabras_local in palabras if palabras_local not in ignorar_letras]
palabras_ = sorted(set(palabras_))

#almacenar las palabras y clases en un archivo binario
pk.dump(palabras_, open('/content/words.pkl','wb'))
pk.dump(clases,open('/content/classes.pkl','wb'))

entrenamiento=[]
output = [0]*len(clases)
for documento in documentos:
  bag=[]
  palabras_patrones = documento[0]
  palabras_patrones = [lematizer.lemmatize(palabras_local.lower()) 
    for palabras_local in palabras_patrones]
  for palabra in palabras_:
    bag.append(1) if palabra in palabras_patrones else bag.append(0)

  output_filas = list(output)
  output_filas[clases.index(documento[1])]=1
  entrenamiento.append([bag,output_filas])
random.shuffle(entrenamiento)
entrenamiento=np.array(entrenamiento)


eje_x = list(entrenamiento[:, 0])
eje_y = list(entrenamiento[:, 1])

#creacion del modelo
model = Sequential() 
model.add(Dense(128, input_shape=(len(eje_x[0]), ), 
                activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(eje_y[0]),  
                activation='softmax')) 

  
# compilar el model 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy', 
             optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(eje_x), np.array(eje_y), 
                 epochs=200, batch_size=5, verbose=1) 
# guardar el model 
model.save("chatbotmodel.h5", hist) 

# imprimir el entrenamiento del modelo

print("exito!") 





<ipython-input-3-29535f3c14ca>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  entrenamiento=np.array(entrenamiento)
/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
5/5 [==============================] - 1s 4ms/step - loss: 2.1589 - accuracy: 0.0000e+00
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 2.1263 - accuracy: 0.0455
Epoch 3/200
5/5 [==============================] - 0s 3ms/step - loss: 1.9467 - accuracy: 0.2727
Epoch 4/200
5/5 [==============================] - 0s 3ms/step - loss: 1.9005 - accuracy: 0.4091
Epoch 5/200
5/5 [==============================] - 0s 3ms/step - loss: 1.9461 - accuracy: 0.2273
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 1.8111 - accuracy: 0.3636
Epoch 7/200
5/5 [==============================] - 0s 5ms/step - loss: 1.6675 - accuracy: 0.4545
Epoch 8/200
5/5 [==============================] - 0s 5ms/step - loss: 1.6274 - accuracy: 0.4091
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 1.6654 - accuracy: 0.5455
Epoch 10/200
5/5 [==============================] - 0s 3ms/step - loss: 1.5842 - accuracy: 0.5909
Epoch 11/200
5/5 [=======

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer2 = nltk.stem.WordNetLemmatizer() 
datas = js.loads(open("data.json").read()) 
words = pk.load(open('words.pkl', 'rb')) 
classes = pk.load(open('classes.pkl', 'rb')) 
model = tf.keras.models.load_model('chatbotmodel.h5') 
  
def clean_up_sentences(sentence): 
    sentence_words = nltk.word_tokenize(sentence) 
    sentence_words = [lemmatizer2.lemmatize(word) 
                      for word in sentence_words] 
                      
    return sentence_words 
  
def bagw(sentence): 
    sentence_words = clean_up_sentences(sentence) 
    bag = [0]*len(words) 
    for w in sentence_words: 
        for i, word in enumerate(words): 
            if word == w: 
                bag[i] = 1
    return np.array(bag) 

def predict_class(sentence): 
    bow = bagw(sentence) 
    res = model.predict(np.array([bow]))[0] 
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) 
               if r > ERROR_THRESHOLD] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_list = [] 
    for r in results: 
        return_list.append({'intent': classes[r[0]], 
                            'probability': str(r[1])}) 
        return return_list 

def get_response(intents_list, intents_json): 
    tag = intents_list[0]['intent'] 
    list_of_intents = intents_json['intents'] 
    result = "" 
    for i in list_of_intents: 
        if i['tag'] == tag: 
            result = random.choice(i['responses']) 
            break
    return result 

print("Chatbot esta ejecutando!") 

  
while True: 
    message = input("") 
    ints = predict_class(message) 
    res = get_response(ints, intents) 
    print(res) 

Chatbot esta ejecutando!
hola
1/1 [==============================] - 0s 96ms/step
¡Hola! ¿Qué necesitas hoy?
que productos estan en ofertas hoy 
1/1 [==============================] - 0s 25ms/step
Claro, aquí te muestro algunos de nuestros productos más populares: taladros, sierras circulares, martillos, cajas de herramientas.
